In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
import keras
from keras.datasets import mnist

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
LATENT_DIMS = [64]
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 3
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 512
ORIGINAL_DIM = 784
results_path = 'results\\mnist_vanillavae_dbal'
epsilon_std = 1.0

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "var_ratios": var_ratios,
    "mean_std": mean_std,
    "max_entropy": max_entropy,
    "bald": bald,
    "uniform": uniform,
    "batch_bald": batch_bald
}


In [4]:
class MLP_REG(nn.Module):
    def __init__(self, latent_dim):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255.
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255.


In [6]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [7]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=98,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [ ]:
for latent_dim in LATENT_DIMS:

    # encoder architecture
    x = Input(shape=(ORIGINAL_DIM,))
    encoder_h = Dense(HIDDEN_DIM, activation='relu')(x)
    z_mean = Dense(latent_dim)(encoder_h)

    # encoder to generate latent variables from input
    encoder = Model(x, z_mean)

    encoder.load_weights(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\mnist\mnist_vanillavae\mnist_vanillavae_z_mean_{latent_dim}.h5".format(latent_dim=latent_dim))
    
    X_train_enhanced = encoder.predict(X_train, batch_size=32)
    X_test_enhanced = encoder.predict(X_test, batch_size=32)
    
    for exp_iter in range(EXPERIMENT_COUNT):
        np.random.seed(exp_iter)
        initial_idx = np.array([],dtype=int)
        for i in range(10):
            idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
            initial_idx = np.concatenate((initial_idx, idx))

        for func_name, acquisition_func in ACQ_FUNCS.items():  
            X_initial = X_train_enhanced[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train_enhanced, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(latent_dim).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test_enhanced,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join(results_path, "{func_name}_latent_dim_{latent_dim}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter, latent_dim=latent_dim))
            np.save(file_name, (acc_arr, dataset_size_arr))

        for func_name, acquisition_func in ACQ_FUNCS.items():  
            X_initial = X_train[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(ORIGINAL_DIM).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join("results\\mnist_dbal", "{func_name}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter))
            np.save(file_name, (acc_arr, dataset_size_arr))


Accuracy after query 1: 0.5814
Accuracy after query 2: 0.6569
Accuracy after query 3: 0.7057
Accuracy after query 4: 0.7339
Accuracy after query 5: 0.7539
Accuracy after query 6: 0.7746
Accuracy after query 7: 0.7885
Accuracy after query 8: 0.7902
Accuracy after query 9: 0.7952
Accuracy after query 10: 0.8218
Accuracy after query 11: 0.8207
Accuracy after query 12: 0.8219
Accuracy after query 13: 0.8490
Accuracy after query 14: 0.8478
Accuracy after query 15: 0.8467
Accuracy after query 16: 0.8589
Accuracy after query 17: 0.8425
Accuracy after query 18: 0.8648
Accuracy after query 19: 0.8741
Accuracy after query 20: 0.8756
Accuracy after query 21: 0.8859
Accuracy after query 22: 0.8918
Accuracy after query 23: 0.8911
Accuracy after query 24: 0.9014
Accuracy after query 25: 0.9086
Accuracy after query 26: 0.9122
Accuracy after query 27: 0.9035
Accuracy after query 28: 0.9039
Accuracy after query 29: 0.9005
Accuracy after query 30: 0.9118
Accuracy after query 31: 0.9173
Accuracy after qu

Accuracy after query 62: 0.9459
Accuracy after query 63: 0.9484
Accuracy after query 64: 0.9495
Accuracy after query 65: 0.9507
Accuracy after query 66: 0.9494
Accuracy after query 67: 0.9479
Accuracy after query 68: 0.9445
Accuracy after query 69: 0.9482
Accuracy after query 70: 0.9493
Accuracy after query 71: 0.9538
Accuracy after query 72: 0.9491
Accuracy after query 73: 0.9442
Accuracy after query 74: 0.9516
Accuracy after query 75: 0.9517
Accuracy after query 76: 0.9533
Accuracy after query 77: 0.9499
Accuracy after query 78: 0.9525
Accuracy after query 79: 0.9518
Accuracy after query 80: 0.9539
Accuracy after query 81: 0.9574
Accuracy after query 82: 0.9616
Accuracy after query 83: 0.9610
Accuracy after query 84: 0.9599
Accuracy after query 85: 0.9599
Accuracy after query 86: 0.9591
Accuracy after query 87: 0.9578
Accuracy after query 88: 0.9598
Accuracy after query 89: 0.9603
Accuracy after query 90: 0.9591
Accuracy after query 91: 0.9575
Accuracy after query 92: 0.9614
Accuracy

Conditional Entropy:   0%|          | 0/2000 [00:00<?, ?it/s]

BatchBALD:   0%|          | 0/10 [00:00<?, ?it/s]

ExactJointEntropy.compute_batch:   0%|          | 0/2000 [00:00<?, ?it/s]

...

BatchBALD:   0%|          | 0/10 [00:00<?, ?it/s]

ExactJointEntropy.compute_batch:   0%|          | 0/2000 [00:00<?, ?it/s]

ExactJointEntropy.compute_batch:   0%|          | 0/2000 [00:00<?, ?it/s]

ExactJointEntropy.compute_batch:   0%|          | 0/2000 [00:00<?, ?it/s]

ExactJointEntropy.compute_batch:   0%|          | 0/2000 [00:00<?, ?it/s]